# MATH 405/607 

# Numerical Methods for Differential Equations

[[Instructor: Christoph Ortner]](http://www.math.ubc.ca/~ortner/)  [[course page]](https://github.com/cortner/math405_2022)

## ODE Boundary Value Problem

or, 2-point bvp.

In [ ]:
include("math405.jl")

### The simplest boundary value problem:

$$\begin{aligned}
- u''(x) + r(x) u(x) &= f(x), \qquad x \in (0, 1) \\ 
  u(0) &= u_0, \\ 
  u(1) &= u_1
\end{aligned}$$

- Does a solution exist? 
- Under which conditions on $r, f$ does a solution exist?
- Is it unique? Stable under perturbations of $r, f$?

One can show that if $f, r$ are "smooth" then a "smooth" solution $u$ exists.

- How do we compute it?

We can make our life particularly easy, 

$$\begin{aligned}
- u''(x) + u(x) &= 1, \qquad x \in (0, 1) \\ 
  u(0) &= 0, \\ 
  u(1) &= 0
\end{aligned}$$

Then we can readily check that the following is a solution:

$$
   u(x) = 1 - \frac{e^x + e^{1-x}}{1+e}
$$

* Question: Why can we get such a simple analytic solution?
* Challenge: is it unique? If yes, provide a proof; if not, provide a counterexample.

In [ ]:
# Plotting the solution 
u(x) = 1 -  (exp(x) + exp(1-x)) / (1 + MathConstants.e)
plot(u, 0, 1, lw=3, label = "u(x)", size = (400, 200))

### Discretising the solution 

We cannot store $u$ in a computer (infinite information), but we could store it (or an approximation) at finitely many points, 

$$
        x_n = n/N, \qquad n = 0, \dots, N.
$$

which are called nodes, mesh, or grid points, etc.
We call $h = 1/N$ the mesh-size or grid-size. We can evaluate $U_n = u(x_n)$ which give us a discretisation of the the solution.

In [ ]:
N = 10; X = range(0, 1, length=N+1)
scatter(X, 0*X, size = (600, 80), label = "grid")
plot!(u, 0, 1, lw=2, label = "continuous solution")
scatter!(X, u.(X), ms=5, label = "discretisation", size = (800, 200), legend=:outertopright)

### Discretising the equation 

But what if we don't know the solution explicitly? This is the *typical* scenario!

The idea is to devise an equation for the *discretised solution*. The first step is to evaluate the equation only at the nodes 

$$
   - u''(x_n) + r(x_n) u(x_n) = f(x_n) 
$$

By evaluating the data $f, r$ at the nodes, $F_n = f(x_n)$ and $R_n = r(x_n)$, we obtain 

$$
  - u''(x_n) + R_n U_n = F_n
  $$
  
What to do about $u''(x_n)$?

### Finite Difference Approximation

Taylor expansion:

$$
    u(x+h) = u(x) + h u'(x) + \frac{h^2}{2} u''(x) + \frac{h^3}{6} u'''(x) + \frac{h^4}{24} u^{(iv)}(\xi)
$$

From this we can deduce, e.g., 

$$\begin{aligned}
    \frac{u(x+h) - u(x)}{h} &= u'(x) + \frac{h}{2} u''(x) + \dots \\ 
    \frac{u(x+h) - 2u(x) + h(x-h)}{h^2} &=  u''(x) + O(h^2)
\end{aligned}$$

Can we replace $u''$ with its FD approximation?

This gives us

$$
    - \frac{u(x_n+h) - 2 u(x_n) + u(x_{n}-h)}{h^2} + r(x_n) u(x_n) = f(x_n), 
$$

But if we choose $h = 1/N$ (we did) then $u(x_n \pm h) = U_{n \pm 1}$, so we can write  a linear system for the *unknowns* $U_n$, $n = 0, \dots, N$:

$$\begin{aligned}
    - \frac{U_{n+1} - 2 U_n + U_{n-1}}{h^2} + R_n U_n  &= F_n, \quad n = 1, \dots, N-1, \\ 
    U_0 &= u_0, \\ 
    U_1 &= u_1.
\end{aligned}$$

(Note for the toy model we just have $u_0 = u_1 = 0, R_n = F_n = 1$.)

### Implementation of the FD Scheme

<!-- $$
\frac{1}{h^2} \begin{pmatrix}
    h^2 &  &  &  &        \\
    -1 & 2 + h^2 & -1  &  &      \\
       & -1 & 2 + h^2 & -1  &     \\
       &    &  \ddots & \ddots & \ddots
\end{pmatrix}
 \cdot 
 \begin{pmatrix}
        U_0 \\ U_1 \\ U_2 \\ \vdots
 \end{pmatrix}
 = 
 \begin{pmatrix}
     0 \\ 1 \\ 1 \\ \vdots 
 \end{pmatrix}
$$
-->

$$
\begin{pmatrix}
    1 &  &  &  &        \\
    - h^{-2} & 2 h^{-2} + 1 & - h^{-2}  &  &      \\
       & -h^{-2} & 2 h^{-2} + 1 & - h^{-2}  &     \\
       &    &  \ddots & \ddots & \ddots
\end{pmatrix}
 \cdot 
 \begin{pmatrix}
        U_0 \\ U_1 \\ U_2 \\ \vdots
 \end{pmatrix}
 = 
 \begin{pmatrix}
     0 \\ 1 \\ 1 \\ \vdots 
 \end{pmatrix}
$$

after eliminating the first and last row (which are trivia) we get the following system:

$$
\begin{pmatrix}
       2 h^{-2} + 1 & - h^{-2}  &  &      \\
       -h^{-2} & 2 h^{-2} + 1 & - h^{-2}  &     \\
          &  \ddots & \ddots & \ddots
\end{pmatrix}
 \cdot 
 \begin{pmatrix}
        U_1 \\ U_2 \\ \vdots
 \end{pmatrix}
 = 
 \begin{pmatrix}
      1 \\ 1 \\ \vdots 
 \end{pmatrix}
$$

$$
\begin{pmatrix}
       2 h^{-2} + 1 & - h^{-2}  &  &      \\
       -h^{-2} & 2 h^{-2} + 1 & - h^{-2}  &     \\
          &  \ddots & \ddots & \ddots
\end{pmatrix}
 \cdot 
 \begin{pmatrix}
        U_1 \\ U_2 \\ \vdots
 \end{pmatrix}
 = 
 \begin{pmatrix}
      1 \\ 1 \\ \vdots 
 \end{pmatrix}
$$

... which we can now implement:

In [ ]:
function assemble_fd(N)
    h = 1/N
    F = ones(N-1)
    A = Tridiagonal( - ones(N-2) / h^2 , 
                     (2/h^2+1)*ones(N-1),
                     - ones(N-2) / h^2 )
    return A, F 
end

In [ ]:
N = 10 
A, F =  assemble_fd(10)
display(A)
display(F)

### Solution of the linear system 

To solve `A * U = F` we use a black box solver provided by Julia, which can be invoked via 
```
U = A \ F
```
Behind the scenes this uses an LU factorisation. Because $A$ is tridiagonal, **AND** we told Julia that it is tri-diagonal the cost of the LU factorisatin will be $O(N)$ only - i.e. quasi-optimal!

In [ ]:
N = 10 
A, F = assemble_fd(N)
U = [ [0.0]; A \ F; [0.0] ] 

X = range(0, 1, length=N+1)
plot(u, 0, 1, lw=3, label = "exact solution", size = (700, 300), legend = :outertopright)
scatter!(X, U, marker = :dot, ms=6, lw=2, ls=:dash, label = "FD approximation")
maxerr = norm(u.(X) - U, Inf)
# println("Error: $(maxerr); this is very good!")
plot!(; title = latexstring("\\max_j  |u(x_j) - U_j| = $(round(maxerr, digits=7))"))

### Error Analysis

We have changed 
- how we represent the solution: a discrete set of points instead of a continuous function;
- how we represent the equation: a finite-difference equation instead of a differential equation.

Naturally, the question arises in what sense we can talk about error, and what that error is. This is one of the major topics of numerical analysis and scientific computing.

In our last computational experiment we have implicitly adopted a seemingly sensible notion of error of our finite-difference solution. And we have shown that it is *remarkably* accurate; ca. 0.1% error which is already far better than typical engineering accuracy. This can be explained rigorously: 


**Theorem:** Let $r > 0$ then 
$$
    \max_{n = 0, \dots, N} |U_n - u(x_n)| \leq \frac{h^2}{96} \| u^{(4)} \|_{\infty}         
$$


But we should now also confirm numerically that our result is (i) correct; and (ii) sharp. With a little bit of basic calculus we can show that $\|u^{(iv)}\|_\infty = 1$. Therefore the estimate becomes $\|E\|_\infty \leq h^2 / 96$.

In [ ]:
get_maxerr(N) = norm( u.(range(0, 1,length=N+1)) - [[0.0]; \(assemble_fd(N)...); [0.0]], Inf )

NN = [8, 16, 32, 64, 128]
plot( NN, get_maxerr.(NN), lw=3, m=:o, ms=6, label = L"\| U - u \|_\infty", 
      xaxis = :log, yaxis = :log, size=(400,300))
plot!( NN, NN.^(-2) / 96, lw=2, ls=:dash, label = "predicted rate" )

### Warning: Advection-Diffusion

Consider an Advection-diffusion problem

$$\begin{aligned}
    - \varepsilon u''(x) + u'(x) = 1, \quad u(0) = u(1) = 0
\end{aligned}$$

discretised via 

$$
    - \varepsilon \frac{U_{n+1} - 2 U_n + U_{n-1}}{h^2} + \frac{U_{n+1} - U_{n-1}}{2h} = 1
$$


In [ ]:
N = 10; epsn = 0.01; o = ones(N-1)  # try epsn = 1, 0.1, 0.01, then N = 30, 100
F = [ [0.0]; o; [0.0] ]
A = Tridiagonal( [ (-epsn*N^2 - 0.5*N) * o; [0.0] ], 
                 [ [1.0]; (2*epsn*N^2) * o; [1.0] ], 
                 [ [0.0]; (- epsn*N^2 + 0.5*N) * o] )
plot(x -> x - (exp(x/epsn)-1)/(exp(1/epsn)-1), 0, 1, lw=3, label = "exact", legend = :topleft)
plot!(range(0, 1, length=N+1), A \ F, lw=3, ms=5, label ="FD approx", size=(400,200))

**Question:** what part of our analysis fails here? 